In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset.v1i.yolov8.zip to dataset.v1i.yolov8.zip


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 622.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import zipfile
import os


zip_file_path = next(iter(uploaded.keys()))  # Get the name of the uploaded file
extract_folder = '/content/dataset'  # Destination folder for extraction

# Create a directory to extract files to
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Check if the dataset has been extracted correctly
print('Extracted files:', os.listdir(extract_folder))
# Extract the uploaded dataset ZIP file

dataset_dir = os.path.join(extract_folder)  # Adjust path if necessary
print('Dataset structure:', os.listdir(dataset_dir))


Extracted files: ['train', 'data.yaml', 'test', 'README.dataset.txt', 'valid', 'README.roboflow.txt']
Dataset structure: ['train', 'data.yaml', 'test', 'README.dataset.txt', 'valid', 'README.roboflow.txt']


In [ ]:
from ultralytics import YOLO

# Load pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')

# Fine-tune the model
model.train(
    data='/content/dataset/data.yaml',
    epochs=100,  # Number of epochs
    imgsz=640,   # Image size
    batch=16,    # Batch size
    lr0=0.01,    # Initial learning rate
    lrf=0.1,     # Final learning rate (multiplier of lr0)
    momentum=0.937,  # Momentum for SGD
    weight_decay=0.0005,  # Weight decay
    warmup_epochs=3,  # Number of warmup epochs
    augment=True,  # Enable data augmentation
    rect=True,     # Rectangular training
    save_period=10  # Save checkpoint every 10 epochs
)

# Evaluate the model
metrics = model.val()

# Extract and print metrics
precision = metrics.box.mp   # Mean precision of all classes
recall = metrics.box.mr      # Mean recall of all classes
map50 = metrics.box.map50    # mAP at IoU 0.5
map50_95 = metrics.box.map   # mAP at IoU 0.5:0.95

# Calculate approximate accuracy (simplified formula)
accuracy = (precision * recall) / (precision + recall) * 2 if (precision + recall) > 0 else 0

# Print metrics
print(f"Mean Precision: {precision:.4f}")
print(f"Mean Recall: {recall:.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")
print(f"Accuracy (approx.): {accuracy:.4f}")

# Optional: Export the trained model in a different format (e.g., ONNX, TorchScript)
model.export(format='onnx')


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.8MB/s]


Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataset/train/labels... 477 images, 0 backgrounds, 0 corrupt: 100%|██████████| 477/477 [00:00<00:00, 1208.56it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


                   all         23         24      0.875      0.833      0.843      0.618
             num_plate          3          3      0.972      0.667      0.717      0.564
          number_plate         21         21      0.777          1      0.969      0.672
Speed: 0.5ms preprocess, 65.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/dataset/valid/labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


                   all         23         24      0.875      0.833      0.843      0.618
             num_plate          3          3      0.972      0.667      0.717      0.564
          number_plate         21         21      0.777          1      0.969      0.672
Speed: 0.5ms preprocess, 73.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train2
Mean Precision: 0.8748
Mean Recall: 0.8333
mAP@0.5: 0.8429
mAP@0.5:0.95: 0.6182
Accuracy (approx.): 0.8535
Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 117.1 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.6s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime 

'runs/detect/train/weights/best.onnx'